In [1]:
import itertools
import sys
sys.path.append("../..")
from graphilp.imports import ilpgraph
from graphilp.network import PCST
from graph_parser import parse_vdf
import networkx as nx
from gurobipy import *
import xlsxwriter

#G = STP_import.stp_file_to_networkx("/mnt/data/projects/ILP/steiner/PUC/bip42u.stp")

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks4.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'cycleBasis', bold)
    worksheet.write('E1', 'nodeColouring', bold)
    worksheet.write('F1', 'Runtime', bold)
    worksheet.write('G1', 'Objective Value', bold)
    worksheet.write('H1', 'Constraints', bold)
    worksheet.write('I1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(settings:list, path:str, linesToRead:range):
    solutions = []
    lines = []
    with open(path) as f:
        formattedFile = f.read().split('\n')
        for i in linesToRead:
            lines.append(formattedFile[i])

    for i in range(len(linesToRead)):
        G, start_terminal, cost_per_meter, prob_id = parse_vdf(lines[i])
        
        Graph = ilpgraph.ILPGraph()
        Graph.setNXGraph(G)
        Graph.setEdgeVars(G.edges())
        Graph.setNodeVars(G.nodes())

        for setting in settings:
            cycleBasis = setting[0]
            nodeColoring = setting[1]
            model = PCST.createModel(Graph, start_terminal, cycleBasis, nodeColoring)
            model.params.MIPGap = 0.05
            model.params.TimeLimit = 600
            if (cycleBasis):
                model.optimize(PCST.callback_cycle) 
            else:
                model.optimize()
            solutions.append((i, len(G.edges()), len(G.nodes()), cycleBasis, nodeColoring, model.runTime, model.objVal,\
                              model.numConstrs, model.MIPGap))
        
    __write_solution(solutions)         

if __name__ == '__main__':
    linesToRead = range(0,3)
    
    #settings = [(False, False), (True,False), (False, True), (True, True)]
    settings = [(True,False), (True, True)]
    path = '/mnt/data/projects/ILP/optimisation_instances_02_he_new.txt'
    
    runBenchmark(settings, path, linesToRead)
    




Using license file /opt/gurobi/gurobi.lic
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 2906 rows, 2316 columns and 12021 nonzeros
Model fingerprint: 0x78fd6668
Variable types: 0 continuous, 2316 integer (1884 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 7e+04]
  Bounds range     [1e+00, 4e+02]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 585 rows and 193 columns
Presolve time: 0.02s
Presolved: 2321 rows, 2123 columns, 9330 nonz

Showing first log only...

Concurrent spin time: 0.09s

Solved with primal simplex

Root relaxation: objective 6.683490e+05, 18043 iterations, 0.97 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 668349.008    0   77 -16883.625 668349.008  4059%     -    1s
H    0     0                    -3072.121083 668349.008      -     -    1s
     0     0 667021.536    0  101 -3072.1211 667021.536      -     -    2s
     0     0 666495.352    0   74 -3072.1211 666495.352      -     -    2s
     0     0 665639.078    0   72 -3072.1211 665639.078      -     -    3s
     0     0 665639.078    0   72 -3072.1211 665639.078      -     -    3s
     0     0 665623.982    0   37 -3072.1211 665623.982      -     -    3s
     0     0 665623.848    0   39 -3072.1211 665623.848      -     -    3s
     0     0 665623.848    0   37 -3072.1211 665623.848      -     -    3s
     0     0 6653

  5759  1566 infeasible   32      623275.825 655702.576  5.20%   208  441s
  5932  1632 infeasible   46      623275.825 655678.056  5.20%   208  453s
  6179  1722 638087.571   34   22 623275.825 655666.862  5.20%   206  465s
  6403  1818 627914.999   38   23 623275.825 655620.407  5.19%   205  476s
  6692  1888 632109.384   37   52 623275.825 655558.550  5.18%   202  486s
  6939  2045     cutoff   33      623275.825 655547.314  5.18%   202  498s
  7249  2196     cutoff   49      623275.825 655521.338  5.17%   199  508s
  7565  2258 640762.196   34   52 623275.825 655216.279  5.12%   197  520s
H 7608  2258                    623275.82954 655216.279  5.12%   197  520s
H 7781  2323                    623275.83891 655216.279  5.12%   196  538s
  8023  2419 infeasible   37      623275.839 655084.826  5.10%   195  549s
  8355  2530 infeasible   39      623275.839 654975.904  5.09%   192  558s
  8650  2651     cutoff   31      623275.839 654725.346  5.05%   192  568s
  8958  2759 infeasible  

  2325   588 infeasible   62      623275.825 658443.596  5.64%   323  246s
  2444   618 649785.678   45   26 623275.825 658306.810  5.62%   319  251s
  2497   681 642342.478   57   61 623275.825 658306.810  5.62%   319  256s
  2584   762 658285.424   58   75 623275.825 658306.742  5.62%   317  261s
  2695   815 632892.325   52   27 623275.825 658306.742  5.62%   315  267s
  2826   865 629948.140   58   94 623275.825 658306.658  5.62%   316  274s
  2914   900 infeasible   48      623275.825 658298.378  5.62%   319  282s
  3023   962 629711.019   47   39 623275.825 658293.491  5.62%   321  290s
  3158  1009 657621.380   49   52 623275.825 658286.312  5.62%   319  300s
  3258  1064 658068.186   41   78 623275.825 658284.025  5.62%   320  310s
  3385  1120     cutoff   59      623275.825 658282.711  5.62%   321  321s
  3529  1188 658242.735   47   34 623275.825 658261.999  5.61%   320  331s
  3690  1230 658054.500   48  101 623275.825 658255.985  5.61%   319  341s
  3868  1271 644447.231  

# Testing Steiner Module

In [3]:
import itertools
import sys
sys.path.append("../..")
from graphilp.imports import ilpgraph,read_stp
from graphilp.network import Steiner
import networkx as nx
from gurobipy import *


def runBenchmark(settings:list, path:str):
    solutions = []
    lines = []


    G, terminals = read_stp.stp_to_networkx(path)

    for setting in settings:
        cycleBasis = setting[0]
        nodeColoring = setting[1]
        model = Steiner.createModel(G, terminals, cycleBasis, nodeColoring)
        model.params.MIPGap = 0.02
        model.params.TimeLimit = 600
        if (cycleBasis):
            model.optimize(Steiner.callback_cycle) 
        else:
            model.optimize()
        solutions.append((len(G.edges()), len(G.nodes()), cycleBasis, nodeColoring, model.runTime, model.objVal,\
                          model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         
    print(solutions)



if __name__ == '__main__':
    
    settings = [(False, False), (True,False), (False, True), (True, True)]
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    
    runBenchmark(settings, path)


Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 5383 rows, 6382 columns and 26492 nonzeros
Model fingerprint: 0xd187df5c
Variable types: 0 continuous, 6382 integer (6382 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 389.0000000
Presolve removed 296 rows and 1453 columns
Presolve time: 0.03s
Presolved: 5087 rows, 4929 columns, 21617 nonzeros
Variable types: 0 continuous, 4929 integer (4929 binary)

Root relaxation: objective 2.317312e+02, 5423 iterations, 0.20 

Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 13347 rows, 6382 columns and 50384 nonzeros
Model fingerprint: 0x734536d2
Variable types: 0 continuous, 6382 integer (6382 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 296 rows and 248 columns
Presolve time: 0.06s
Presolved: 13051 rows, 6134 columns, 45514 nonzeros
Variable types: 0 continuous, 6134 integer (6134 binary)
Found heuristic solution: objective 294.0000000

Root relaxation: objective 2.317312e+02, 10118 iterations, 0.7

In [7]:
import itertools
import sys
sys.path.append("../..")
from graphilp.imports import ilpgraph,read_stp
from graphilp.network import Steiner_Linear
import networkx as nx
from gurobipy import *


def runBenchmark(settings:list, path:str):
    solutions = []
    lines = []


    G, terminals = read_stp.stp_to_networkx(path)
    model = Steiner_Linear.createModel(G, terminals)
    model.params.MIPGap = 0.02
    model.params.TimeLimit = 600
    model.optimize()
    solutions.append((len(G.edges()), len(G.nodes()), cycleBasis, nodeColoring, model.runTime, model.objVal,\
                      model.numConstrs, model.MIPGap))
        
    #__write_solution(solutions)         
    print(solutions)



if __name__ == '__main__':
    
    path = '/mnt/data/projects/ILP/steiner/PUC/bip42u.stp'
    runBenchmark(settings, path)


Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter MIPGap to 0.02
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 3 threads
Optimize a model with 18529 rows, 10364 columns and 90204 nonzeros
Model fingerprint: 0xaae93009
Variable types: 0 continuous, 10364 integer (9164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 4278 rows and 248 columns
Presolve time: 0.10s
Presolved: 14251 rows, 10116 columns, 70262 nonzeros
Variable types: 0 continuous, 10116 integer (8916 binary)

Root relaxation: objective 2.318806e+02, 13664 iterations, 1.55 seconds

    Nodes    |    Current Node  

NameError: name 'cycleBasis' is not defined